In [1]:
import vertexai

PROJECT_ID = "adventml"  # @param {type:"string"}
vertexai.init(project=PROJECT_ID, location="us-central1")

/usr/local/lib/python3.8/dist-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
/root/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from typing import Any, List, Tuple, Union
import re

from langchain import LLMChain
from langchain.agents import (
    AgentExecutor,
    BaseSingleActionAgent,
    LLMSingleActionAgent,
    AgentOutputParser,
    Tool
)
from langchain.agents.chat.prompt import FORMAT_INSTRUCTIONS
from langchain.embeddings import VertexAIEmbeddings
from langchain.embeddings.base import Embeddings
from langchain.llms.vertexai import VertexAI
from langchain.prompts import StringPromptTemplate
from langchain.schema import AgentAction, AgentFinish
from langchain.tools import BaseTool
from langchain.tools.base import tool


In [5]:
# LLM model
llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

In [7]:
class MyVertexAIEmbedding(VertexAIEmbeddings, Embeddings):
    model_name: str = "textembedding-gecko"
    max_batch_sizes: int = 5

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Embed a list of strings.

        Args:
            texts: List[str] The list of strings to embed.

        Returns:
            List of embeddings, one for each text.
        """
        chunked_texts = [
            texts[i : i + self.max_batch_sizes]
            for i in range(0, len(texts), self.max_batch_sizes)
        ]
        embeddings = []

        for chunk in chunked_texts:
            embeddings.extend(self.client.get_embeddings(chunk))

        return [el.values for el in embeddings]

    def embed_query(self, text: str) -> List[float]:
        """Embed a text.

        Args:
            text: The text to embed.

        Returns:
            Embedding for the text.
        """
        embeddings = self.client.get_embeddings([text])
        return embeddings[0].values

In [8]:
embeddings = MyVertexAIEmbedding()

In [9]:
class MockTool(BaseTool):
    name = "MockTool"
    description = "useful for when you need to answer questions about a person"

    def _run(self, query: str) -> str:
        """Use the tool."""
        print(f"*** Invoking MockTool with query '{query}'")
        return f"Answer of '{query}' is 'Michael Chi'"

    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        print(f"*** Invoking MockTool with query '{query}'")
        return f"Answer of '{query}' is 'Michael Chi'"


# return_direct=True | False
@tool("MockTool_Function", return_direct=False)
def MockTool_Function(query: str) -> str:
    """
    useful for when you need to answer questions about everything about Google PaLM2
    """
    print(f">MockTool_Functioon says {query}")
    return """
        PaLM 2 is our next generation large language model that builds on Google’s legacy
        of breakthrough research in machine learning and responsible AI.
        It excels at advanced reasoning tasks, including code and math,
        classification and question answering, translation and multilingual proficiency,
        and natural language generation better than our previous state-of-the-art LLMs,
        including PaLM. It can accomplish these tasks because of the way it was built
        – bringing together compute-optimal scaling, an improved dataset mixture,
        and model architecture improvements.
        PaLM 2 is grounded in Google’s approach to building and deploying AI responsibly.
        It was evaluated rigorously for its potential harms and biases,
        capabilities and downstream uses in research and in-product applications.

        It’s being used in other state-of-the-art models, like Med-PaLM 2 and Sec-PaLM,
        and is powering generative AI features and tools at Google,
        like Bard and the PaLM API.
        """


@tool("MockTool_Function_Return_Direct", return_direct=True)
def MockTool_Function_Return_Direct(query: str) -> str:
    """
    useful for when you need to answer questions about everything about Google Bard
    """
    return """
        Bard is an experiment based on this same technology that lets you collaborate with generative AI.
        As a creative and helpful collaborator, Bard can supercharge your imagination, boost your productivity,
        and help you bring your ideas to life.

        If you’re interested in the more technical details, LaMDA is a Transformer-based model,
        the machine-learning breakthrough invented by Google in 2017Opens in a new window.

        The language model learns by “reading” trillions of words that help it pick up on patterns that make up human language,
        so it’s good at predicting what might be reasonable responses.
        """




class VertexPromptTemplate(StringPromptTemplate):
    template: str = """
        You are an helpful agent.
        Answer the following questions as best you can.
        You have access to the following tools:

        {tools}

        Use the following format:

        Question: the input question you must answer
        Thought: you should alJeeways think about what to do
        Action: the action to take, should be one of [{tool_names}]
        Action Input: the input to the action
        Observation: the result of the action
        ... (this Thought/Action/Action Input/Observation can repeat N times)
        Thought: I now know the final answer
        Final Answer: the final answer to the original input question.

        Begin!

        Question: {input}
        {agent_scratchpad}"""
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        intermediate_steps = kwargs.pop("intermediate_steps")
        # This is the first iteration, there is no agent thoughts yet.
        if "agent_scratchpad" not in kwargs:
            kwargs["agent_scratchpad"] = ""
        # Retrieve action and observation of this iteration
        for action, observation in intermediate_steps:
            kwargs["agent_scratchpad"] += action.log
            kwargs["agent_scratchpad"] += f"\nObservation: {observation}\nThought: "
        kwargs["tools"] = "\n".join(
            [f"* {tool.name}: {tool.description}" for tool in self.tools]
        )
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)


class VertexLLMOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        print("***** VertexLLMOutputParser::parse()::llm_output->{}".format(llm_output))
        # If we have the final answer
        if "Final Answer:" in llm_output:
            return AgentFinish(
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # The output should be in the following format
        # Action: "action name"
        # Action Input: "inputs"
        # Sometimes the LLM may produce Action Input without double quotes
        # We need to handle this.
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input:$"
        match = re.search(regex, llm_output, re.DOTALL)
        if match:
            llm_output = llm_output + "\"\""

        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        return AgentAction(
            tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output
        )

In [11]:
mock = MockTool(verbose=True)
mock_func = MockTool_Function
tools = [
    Tool(
        name=mock.name,
        func=mock.run,
        description=mock.description,
        # return_direct=True  # I want to return the result directly to the user
    ),
    mock_func,
]

print(tools)

[Tool(name='MockTool', description='useful for when you need to answer questions about a person', func=<bound method BaseTool.run of MockTool(verbose=True)>), StructuredTool(name='MockTool_Function', description='MockTool_Function(query: str) -> str - useful for when you need to answer questions about everything about Google PaLM2', args_schema=<class 'pydantic.v1.main.MockTool_FunctionSchemaSchema'>, func=<function MockTool_Function at 0x7f86ffc6e940>)]


In [12]:
prompt = VertexPromptTemplate(
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"],
)


llm_chain = LLMChain(llm=llm, prompt=prompt)

output_parser = VertexLLMOutputParser()
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names,
)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

result = agent_executor.run(input="Who is the speaker of this session ?")
print("*****")
print(result)



> Entering new AgentExecutor chain...
***** VertexLLMOutputParser::parse()::llm_output->Thought: I need to know who is the speaker of this session
Action: MockTool
Action Input: 
Thought: I need to know who is the speaker of this session
Action: MockTool
Action Input: *** Invoking MockTool with query ''
Answer of '' is 'Michael Chi'

Observation:Answer of '' is 'Michael Chi'
***** VertexLLMOutputParser::parse()::llm_output->I now know the final answer
Final Answer: Michael Chi
I now know the final answer
Final Answer: Michael Chi

> Finished chain.
*****
Michael Chi


In [13]:

"""
Agent Class
"""


class MyCustomAgent(BaseSingleActionAgent):
    """My Custom Agent."""

    @property
    def input_keys(self):
        return ["input"]

    def plan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[AgentAction, AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        return AgentAction(tool="Search", tool_input=kwargs["input"], log="")

    async def aplan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[AgentAction, AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        return AgentAction(tool="Search", tool_input=kwargs["input"], log="")


"""
Run
"""
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=MyCustomAgent(), tools=tools, verbose=True
)
result = agent_executor.run("How many people live in taiwan as of 2023?")

print(result)



> Entering new AgentExecutor chain...
Search is not a valid tool, try one of [MockTool, MockTool_Function].Search is not a valid tool, try one of [MockTool, MockTool_Function].Search is not a valid tool, try one of [MockTool, MockTool_Function].Search is not a valid tool, try one of [MockTool, MockTool_Function].Search is not a valid tool, try one of [MockTool, MockTool_Function].Search is not a valid tool, try one of [MockTool, MockTool_Function].Search is not a valid tool, try one of [MockTool, MockTool_Function].Search is not a valid tool, try one of [MockTool, MockTool_Function].Search is not a valid tool, try one of [MockTool, MockTool_Function].Search is not a valid tool, try one of [MockTool, MockTool_Function].Search is not a valid tool, try one of [MockTool, MockTool_Function].Search is not a valid tool, try one of [MockTool, MockTool_Function].Search is not a valid tool, try one of [MockTool, MockTool_Function].Search is not a valid tool, try one of [MockTool, MockTool_Func